![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/join.png)

# Join

In Data Prep you can easily join two Dataflows.

In [ ]:
import azureml.dataprep as dprep

First, get the left side of the data into a shape that is ready for the join.

In [ ]:
# get the first Dataflow and derive desired key column
dflow_left = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/testfiles/BostonWeather.csv')
dflow_left = dflow_left.derive_column_by_example(source_columns='DATE', new_column_name='date_timerange',
                                                 example_data=[('11/11/2015 0:54', 'Nov 11, 2015 | 12AM-2AM'),
                                                              ('2/1/2015 0:54', 'Feb 1, 2015 | 12AM-2AM'),
                                                              ('1/29/2015 20:54', 'Jan 29, 2015 | 8PM-10PM')])
dflow_left = dflow_left.drop_columns(['DATE'])

# convert types and summarize data
dflow_left = dflow_left.set_column_types(type_conversions={'HOURLYDRYBULBTEMPF': dprep.TypeConverter(dprep.FieldType.DECIMAL)})
dflow_left = dflow_left.filter(expression=~dflow_left['HOURLYDRYBULBTEMPF'].is_error())
dflow_left = dflow_left.summarize(group_by_columns=['date_timerange'],summary_columns=[dprep.SummaryColumnsValue('HOURLYDRYBULBTEMPF', dprep.api.engineapi.typedefinitions.SummaryFunction.MEAN, 'HOURLYDRYBULBTEMPF_Mean')] )

# cache the result so the steps above are not executed every time we pull on the data
import os
from pathlib import Path
cache_dir = str(Path(os.getcwd(), 'dataflow-cache'))
dflow_left.cache(directory_path=cache_dir)
dflow_left.head(5)

Now let's prepare the data for the right side of the join.

In [ ]:
# get the second Dataflow and desired key column
dflow_right = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/bike-share/*-hubway-tripdata.csv')
dflow_right = dflow_right.keep_columns(['starttime', 'start station id'])
dflow_right = dflow_right.derive_column_by_example(source_columns='starttime', new_column_name='l_date_timerange',
                                                 example_data=[('2015-01-01 00:21:44', 'Jan 1, 2015 | 12AM-2AM')])
dflow_right = dflow_right.drop_columns('starttime')

# cache the results
dflow_right.cache(directory_path=cache_dir)
dflow_right.head(5)

There are three ways you can join two Dataflows in Data Prep:
1. Create a `JoinBuilder` object for interactive join configuration.
2. Call ```join()``` on one of the Dataflows and pass in the other along with all other arguments.
3. Call ```Dataflow.join()``` method and pass in two Dataflows along with all other arguments.

We will explore the builder object as it simplifies the determination of correct arguments. 

In [ ]:
# construct a builder for joining dataflow_l with dataflow_r
join_builder = dflow_left.builders.join(right_dataflow=dflow_right, left_column_prefix='l', right_column_prefix='r')

join_builder

So far the builder has no properties set except default values.
From here you can set each of the options and preview its effect on the join result or use Data Prep to determine some of them.

Let's start with determining appropriate column prefixes for left and right side of the join and lists of columns that would not conflict and therefore don't need to be prefixed.

In [ ]:
join_builder.detect_column_info()
join_builder

You can see that Data Prep has performed a pull on both Dataflows to determine the column names in them. Given that `dataflow_r` already had a column starting with `l_` new prefix got generated which would not collide with any column names that are already present.
Additionally columns in each Dataflow that won't conflict during join would remain unprefixed.
This apprach to column naming is crucial for join robustness to schema changes in the data. Let's say that at some time in future the data consumed by left Dataflow will also have `l_date_timerange` column in it.
Configured as above the join will still run as expected and the new column will be prefixed with `l2_` ensuring that ig column `l_date_timerange` was consumed by some other future transformation it remains unaffected.

Note: `KEY_generated` is appended to both lists and is reserved for Data Prep use in case Autojoin is performed.

### Autojoin
Autojoin is a Data prep feature that determines suitable join arguments given data on both sides. In some cases Autojoin can even derive a key column from a number of available columns in the data.
Here is how you can use Autojoin:

In [ ]:
# generate join suggestions
join_builder.generate_suggested_join()

# list generated suggestions
join_builder.list_join_suggestions()

Now let's select the first suggestion and preview the result of the join.

In [ ]:
# apply first suggestion
join_builder.apply_suggestion(0)

join_builder.preview(10)

Now, get our new joined Dataflow.

In [ ]:
dflow_autojoined = join_builder.to_dataflow().drop_columns(['l_date_timerange'])

### Joining two Dataflows without pulling the data

If you don't want to pull on data and know what join should look like, you can always use the join method on the Dataflow.

In [ ]:
dflow_joined = dprep.Dataflow.join(left_dataflow=dflow_left,
                                      right_dataflow=dflow_right,
                                      join_key_pairs=[('date_timerange', 'l_date_timerange')],
                                      left_column_prefix='l2_',
                                      right_column_prefix='r_')


In [ ]:
dflow_joined.head(5)

In [ ]:
dflow_joined = dflow_joined.filter(expression=dflow_joined['r_start station id'] == '67')
df = dflow_joined.to_pandas_dataframe()
df